# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 1:15PM,237659,20,7995914-BO,Reliable 1 Laboratories LLC,Released
1,May 19 2022 1:15PM,237659,20,7978738-BO,Reliable 1 Laboratories LLC,Released
2,May 19 2022 1:15PM,237659,20,8000494-BO,Reliable 1 Laboratories LLC,Released
3,May 19 2022 1:15PM,237659,20,8005616-BO,Reliable 1 Laboratories LLC,Released
4,May 19 2022 1:15PM,237659,20,8005615-BO,Reliable 1 Laboratories LLC,Released
5,May 19 2022 1:15PM,237659,20,8005617-BO,Reliable 1 Laboratories LLC,Released
6,May 19 2022 1:15PM,237659,20,8005618-BO,Reliable 1 Laboratories LLC,Released
7,May 19 2022 1:15PM,237659,20,8027913,Reliable 1 Laboratories LLC,Released
8,May 19 2022 1:15PM,237659,20,8032393,Reliable 1 Laboratories LLC,Released
9,May 19 2022 1:15PM,237659,20,7965765-BO,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,237656,Released,1
41,237657,Released,13
42,237658,Released,1
43,237659,Released,15
44,237660,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237656,NaN,NaN,1.0
237657,NaN,NaN,13.0
237658,NaN,NaN,1.0
237659,NaN,NaN,15.0
237660,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237566,0.0,0.0,4.0
237567,0.0,0.0,1.0
237571,0.0,1.0,0.0
237575,0.0,1.0,0.0
237583,0.0,0.0,25.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237566,0,0,4
237567,0,0,1
237571,0,1,0
237575,0,1,0
237583,0,0,25


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237566,0,0,4
1,237567,0,0,1
2,237571,0,1,0
3,237575,0,1,0
4,237583,0,0,25


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237566,,,4
1,237567,,,1
2,237571,,1,
3,237575,,1,
4,237583,,,25


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 1:15PM,237659,20,Reliable 1 Laboratories LLC
15,May 19 2022 1:13PM,237660,19,"Avet Pharmaceuticals Labs, Inc"
16,May 19 2022 1:12PM,237657,20,Alumier Labs Inc.
29,May 19 2022 1:11PM,237658,19,"Quagen Pharmaceuticals, LLC"
30,May 19 2022 1:06PM,237656,19,"AdvaGen Pharma, Ltd"
31,May 19 2022 1:01PM,237655,20,"ACI Healthcare USA, Inc."
36,May 19 2022 12:55PM,237654,17,Twinlab Consolidated Corporation
37,May 19 2022 12:46PM,237653,10,"Nextsource Biotechnology, LLC"
39,May 19 2022 12:43PM,237652,21,"NBTY Global, Inc."
40,May 19 2022 12:25PM,237651,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 1:15PM,237659,20,Reliable 1 Laboratories LLC,,,15
1,May 19 2022 1:13PM,237660,19,"Avet Pharmaceuticals Labs, Inc",,,1
2,May 19 2022 1:12PM,237657,20,Alumier Labs Inc.,,,13
3,May 19 2022 1:11PM,237658,19,"Quagen Pharmaceuticals, LLC",,,1
4,May 19 2022 1:06PM,237656,19,"AdvaGen Pharma, Ltd",,,1
5,May 19 2022 1:01PM,237655,20,"ACI Healthcare USA, Inc.",,,5
6,May 19 2022 12:55PM,237654,17,Twinlab Consolidated Corporation,,,1
7,May 19 2022 12:46PM,237653,10,"Nextsource Biotechnology, LLC",,,2
8,May 19 2022 12:43PM,237652,21,"NBTY Global, Inc.",,,1
9,May 19 2022 12:25PM,237651,19,"AdvaGen Pharma, Ltd",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 1:15PM,237659,20,Reliable 1 Laboratories LLC,15,,
1,May 19 2022 1:13PM,237660,19,"Avet Pharmaceuticals Labs, Inc",1,,
2,May 19 2022 1:12PM,237657,20,Alumier Labs Inc.,13,,
3,May 19 2022 1:11PM,237658,19,"Quagen Pharmaceuticals, LLC",1,,
4,May 19 2022 1:06PM,237656,19,"AdvaGen Pharma, Ltd",1,,
5,May 19 2022 1:01PM,237655,20,"ACI Healthcare USA, Inc.",5,,
6,May 19 2022 12:55PM,237654,17,Twinlab Consolidated Corporation,1,,
7,May 19 2022 12:46PM,237653,10,"Nextsource Biotechnology, LLC",2,,
8,May 19 2022 12:43PM,237652,21,"NBTY Global, Inc.",1,,
9,May 19 2022 12:25PM,237651,19,"AdvaGen Pharma, Ltd",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 1:15PM,237659,20,Reliable 1 Laboratories LLC,15,,
1,May 19 2022 1:13PM,237660,19,"Avet Pharmaceuticals Labs, Inc",1,,
2,May 19 2022 1:12PM,237657,20,Alumier Labs Inc.,13,,
3,May 19 2022 1:11PM,237658,19,"Quagen Pharmaceuticals, LLC",1,,
4,May 19 2022 1:06PM,237656,19,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 1:15PM,237659,20,Reliable 1 Laboratories LLC,15.0,NaN,NaN
1,May 19 2022 1:13PM,237660,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN,NaN
2,May 19 2022 1:12PM,237657,20,Alumier Labs Inc.,13.0,NaN,NaN
3,May 19 2022 1:11PM,237658,19,"Quagen Pharmaceuticals, LLC",1.0,NaN,NaN
4,May 19 2022 1:06PM,237656,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 1:15PM,237659,20,Reliable 1 Laboratories LLC,15.0,0.0,0.0
1,May 19 2022 1:13PM,237660,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0,0.0
2,May 19 2022 1:12PM,237657,20,Alumier Labs Inc.,13.0,0.0,0.0
3,May 19 2022 1:11PM,237658,19,"Quagen Pharmaceuticals, LLC",1.0,0.0,0.0
4,May 19 2022 1:06PM,237656,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2376341,127.0,29.0,0.0
16,475203,2.0,0.0,0.0
17,237654,1.0,0.0,0.0
18,2376080,17.0,1.0,0.0
19,1425869,11.0,19.0,19.0
20,1188140,80.0,0.0,0.0
21,475227,1.0,1.0,0.0
22,950491,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2376341,127.0,29.0,0.0
1,16,475203,2.0,0.0,0.0
2,17,237654,1.0,0.0,0.0
3,18,2376080,17.0,1.0,0.0
4,19,1425869,11.0,19.0,19.0
5,20,1188140,80.0,0.0,0.0
6,21,475227,1.0,1.0,0.0
7,22,950491,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,127.0,29.0,0.0
1,16,2.0,0.0,0.0
2,17,1.0,0.0,0.0
3,18,17.0,1.0,0.0
4,19,11.0,19.0,19.0
5,20,80.0,0.0,0.0
6,21,1.0,1.0,0.0
7,22,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,127.0
1,16,Released,2.0
2,17,Released,1.0
3,18,Released,17.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
Executing,29.0,0.0,0.0,1.0,19.0,0.0,1.0,1.0
Released,127.0,2.0,1.0,17.0,11.0,80.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
1,Executing,29.0,0.0,0.0,1.0,19.0,0.0,1.0,1.0
2,Released,127.0,2.0,1.0,17.0,11.0,80.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
1,Executing,29.0,0.0,0.0,1.0,19.0,0.0,1.0,1.0
2,Released,127.0,2.0,1.0,17.0,11.0,80.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()